In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import requests
import pymongo
import time

## Mars News

In [3]:
# Splinter
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
# URL of page to be scraped
news_url = 'https://mars.nasa.gov/news/'

browser.visit(news_url)
html = browser.html

# Create BeautifulSoup object
news_soup = bs(html, "html.parser")

In [5]:
# Retrieve news titles and paragraph texts

time.sleep(2)
news_title = news_soup.select('.content_title')[0].text
news_p = news_soup.select('.article_teaser_body')[0].text
        
# Run if title and paragraph are available
if (news_title and news_p):
    print(news_title)
    print(news_p)

NASA's MRO Completes 60,000 Trips Around Mars
The orbiting spacecraft is also about to set a record for data relayed from the Martian surface.


In [6]:
browser.quit()

## JPL Mars Space Images - Featured Image

In [7]:
# URL to be scraped
images_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

# Use splinter to navigate the site and find the image url for the current 
# Featured Mars Image and assign the url string to a variable called featured_image_url
executable_path = {'executable_path': '/Users/lucyly/Downloads/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

browser.visit(images_url)

In [9]:
browser.find_by_css('ul.articles')

image_soup = bs(browser.html, 'html.parser')
image_src = image_soup.select('a.fancybox')[0]['data-fancybox-href']
print(image_src)

/spaceimages/images/mediumsize/PIA18851_ip.jpg


In [10]:
featured_image_url = 'https://www.jpl.nasa.gov/' + image_src
print(featured_image_url)

https://www.jpl.nasa.gov//spaceimages/images/mediumsize/PIA18851_ip.jpg


In [11]:
browser.quit()

## Mars Weather

In [12]:
# URL of page to be scraped
weather_url = 'https://twitter.com/marswxreport?lang=en'

# Retrieve page with the requests module
weather_response = requests.get(weather_url)
# Create BeautifulSoup object
weather_soup = bs(weather_response.text)

In [13]:
# Retrieve weather tweet texts
mars_weather_tweet = weather_soup.select('div.js-tweet-text-container')

In [14]:
# Print tweets and add to MongoDB
for tweet in mars_weather_tweet: 
    if tweet.text.strip().startswith('InSight'):
        mars_weather = tweet.text.strip()
        print(mars_weather)

InSight sol 167 (2019-05-17) low -100.5ºC (-148.9ºF) high -20.4ºC (-4.6ºF)
winds from the SW at 4.7 m/s (10.6 mph) gusting to 13.5 m/s (30.3 mph)
pressure at 7.50 hPapic.twitter.com/0Eqt9nN21o
InSight sol 166 (2019-05-15) low -100.5ºC (-148.8ºF) high -20.5ºC (-4.8ºF)
winds from the W at 4.2 m/s (9.4 mph) gusting to 11.7 m/s (26.2 mph)
pressure at 7.50 hPa
InSight sol 165 (2019-05-15) low -100.3ºC (-148.6ºF) high -18.2ºC (-0.7ºF)
winds from the SW at 4.6 m/s (10.4 mph) gusting to 13.7 m/s (30.6 mph)
pressure at 7.50 hPapic.twitter.com/7NMgdAkFA8
InSight sol 164 (2019-05-13) low -100.0ºC (-147.9ºF) high -16.6ºC (2.1ºF)
winds from the W at 4.1 m/s (9.1 mph) gusting to 15.1 m/s (33.7 mph)
pressure at 7.50 hPapic.twitter.com/XqV9CJUUoa
InSight sol 163 (2019-05-13) low -99.9ºC (-147.7ºF) high -17.7ºC (0.2ºF)
winds from the SW at 4.3 m/s (9.7 mph) gusting to 15.2 m/s (34.0 mph)
pressure at 7.50 hPapic.twitter.com/qtElTnSRJj
InSight sol 162 (2019-05-12) low -100.2ºC (-148.3ºF) high -20.3ºC (-4

## Mars Facts

In [15]:
# URL to be scraped 
facts_url = 'https://space-facts.com/mars/'

In [16]:
#  use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
mars_df = pd.read_html(facts_url)
mars_df = mars_df[0]
mars_df.columns = ['Description', 'Value']
mars_df.set_index('Description', inplace=True)
mars_df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [17]:
# Use Pandas to convert the data to a HTML table string.
mars_html = mars_df.to_html()
mars_html = mars_html.replace('\n', ' ')
mars_html

'<table border="1" class="dataframe">   <thead>     <tr style="text-align: right;">       <th></th>       <th>Value</th>     </tr>     <tr>       <th>Description</th>       <th></th>     </tr>   </thead>   <tbody>     <tr>       <th>Equatorial Diameter:</th>       <td>6,792 km</td>     </tr>     <tr>       <th>Polar Diameter:</th>       <td>6,752 km</td>     </tr>     <tr>       <th>Mass:</th>       <td>6.42 x 10^23 kg (10.7% Earth)</td>     </tr>     <tr>       <th>Moons:</th>       <td>2 (Phobos &amp; Deimos)</td>     </tr>     <tr>       <th>Orbit Distance:</th>       <td>227,943,824 km (1.52 AU)</td>     </tr>     <tr>       <th>Orbit Period:</th>       <td>687 days (1.9 years)</td>     </tr>     <tr>       <th>Surface Temperature:</th>       <td>-153 to 20 °C</td>     </tr>     <tr>       <th>First Record:</th>       <td>2nd millennium BC</td>     </tr>     <tr>       <th>Recorded By:</th>       <td>Egyptian astronomers</td>     </tr>   </tbody> </table>'

## Mars Hemispheres

In [18]:
# Splinter
executable_path = {'executable_path': '/Users/lucyly/Downloads/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [19]:
# URL to be scraped: 
hemi_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(hemi_url)
html = browser.html

In [20]:
# Save both the image url string for the full resolution hemisphere image, 
# and the Hemisphere title containing the hemisphere name
hemisphere_image_urls = []

for i in range (4):
    time.sleep(3)
    images = browser.find_by_tag('h3')
    images[i].click()
    html = browser.html
    soup = bs(html, 'html.parser')
    time.sleep(3)
    partial = soup.find("img", class_="wide-image")["src"]
    img_title = soup.find("h2",class_="title").text
    img_url = 'https://astrogeology.usgs.gov'+ partial
    hemi_dict={"title":img_title,"img_url":img_url}
    hemisphere_image_urls.append(hemi_dict)
    browser.back()

In [21]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]

In [22]:
browser.quit()